In [43]:
from query.models import Face, Shot
from rekall.video_interval_collection import VideoIntervalCollection
from rekall.parsers import named_payload, in_array, bbox_payload_parser
from rekall.parsers import merge_dict_parsers, dict_payload_parser
from rekall.merge_ops import payload_plus, payload_first, merge_named_payload
from rekall.payload_predicates import payload_satisfies, on_name
from rekall.spatial_predicates import scene_graph
from rekall.logical_predicates import and_pred
from rekall.temporal_predicates import overlaps
from rekall.bbox_predicates import height_at_least, left_of, same_value
from esper.rekall import intrvllists_to_result_with_objects, bbox_to_result_object

In [6]:
from esper.prelude import esper_widget
import time
from IPython.display import display

# Query

In [41]:
def query(video_id, MIN_FACE_HEIGHT, MIN_BRIGHTNESS, stride):
    # We're going to look for frames that would be good "hero shot" frames --
    #   potentially good frames to show in a Netflix preview, for instance.
    # We're going to look for frames where there's exactly one face of a
    #   certain height, and the frame has certain minimum brightness,
    #   sharpness, and contrast properties.
#     MIN_FACE_HEIGHT = 0.2
#     MIN_BRIGHTNESS = 50

    # Annotate face rows with start and end frames, video ID, and frame image
    #   information
    faces_qs = Face.objects.annotate(
        min_frame=F('frame__number'),
        max_frame=F('frame__number'),
        video_id=F('frame__video_id'),
        brightness=F('frame__brightness')
    ).filter(
        frame__video_id=video_id,
        brightness__isnull=False
    )

    # Load bounding boxes and faces into rekall, and put all faces in one frame
    faces = VideoIntervalCollection.from_django_qs(
        faces_qs,
        with_payload=merge_dict_parsers([named_payload('faces',
            in_array(bbox_payload_parser(VideoIntervalCollection.django_accessor))),
            dict_payload_parser(VideoIntervalCollection.django_accessor, {
                'brightness': 'brightness'
            })])
    ).coalesce(merge_named_payload({
        'faces': payload_plus,
        'brightness': payload_first
    }))

    # Hero shots are shots where there is exactly one face of at least a
    #   certain height, and brightness, contrast, and sharpness are at least
    #   some amount
    hero_shot_frames = faces.filter(payload_satisfies(and_pred(
        on_name('faces', scene_graph({
            'nodes': [{ 'name': 'face', 'predicates': [
                height_at_least(MIN_FACE_HEIGHT) ] }],
            'edges': []
        }, exact=True)),
        lambda payload: (payload['brightness'] > MIN_BRIGHTNESS)
    )))
    
    shots_qs = Shot.objects.filter(
        video_id=video_id,
        cinematic=True)
    shots = VideoIntervalCollection.from_django_qs(shots_qs)
    
    hero_shots = shots.filter_against(hero_shot_frames, predicate=overlaps())
    
    return intrvllists_to_result_with_objects(
        hero_shots.get_allintervals(), 
        lambda a, b: [],
#         lambda payload, video_id: [
#             bbox_to_result_object(bbox, video_id) for bbox in payload['faces']],
        stride=stride)

In [39]:
def show_query(video_id, min_face_height, min_brightness, stride):
    result = query(video_id, min_face_height, min_brightness, stride)
    
    widget = esper_widget(result, jupyter_keybindings=True, crop_bboxes=False)
    display(widget)
    
    return widget, result

In [79]:
def convert_segments(segments):
    output_segments = []
    for seg in segments:
        for res in result['result']:
            if res['elements'][0]['min_frame'] == seg[0]:
                output_segments.append((res['elements'][0]['min_frame'], res['elements'][0]['max_frame']))
    print(output_segments)

# Braveheart

In [82]:
widget, result = show_query(28, 0.2, 75, 10)

VGridWidget(jsglobals={'schema': [['Identity', ['id', 'name']], ['Genre', ['id', 'name']], ['Director', ['id',…

In [46]:
selected_segments_braveheart = [
    (result['result'][i]['elements'][0]['min_frame'], result['result'][i]['elements'][0]['min_frame'])
    for i in widget.selected
]
print(selected_segments_braveheart)

[(48232, 48232), (72574, 72574), (108792, 108792), (111688, 111688), (115709, 115709), (116403, 116403), (122525, 122525), (132329, 132329), (153664, 153664), (232645, 232645)]


In [78]:
convert_segments(selected_segments_braveheart)

[(48232, 48300), (72574, 72649), (108792, 108835), (111688, 111995), (115709, 115730), (116403, 116498), (122525, 122543), (132329, 132383), (153664, 153720), (232645, 232673)]


# Revenge of the Sith

In [47]:
start = time.time()

In [80]:
widget, result = show_query(186, 0.2, 50, 5)

VGridWidget(jsglobals={'schema': [['Identity', ['id', 'name']], ['Genre', ['id', 'name']], ['Director', ['id',…

In [51]:
selected_segments_rots = [
    (result['result'][i]['elements'][0]['min_frame'], result['result'][i]['elements'][0]['min_frame'])
    for i in widget.selected
]
print(selected_segments_rots)

[(13733, 13733), (25961, 25961), (28038, 28038), (34901, 34901), (35493, 35493), (49620, 49620), (60137, 60137), (72843, 72843), (73402, 73402), (78752, 78752), (92949, 92949), (130785, 130785), (181722, 181722), (186324, 186324)]


In [81]:
convert_segments(selected_segments_rots)

[(13733, 13796), (25961, 26277), (28038, 28096), (34901, 34961), (35493, 35568), (49620, 49689), (60137, 60240), (72843, 72982), (73402, 73534), (78752, 78800), (92949, 92990), (130785, 131036), (181722, 181794), (186324, 186429)]


In [52]:
end = time.time()

In [53]:
print("Seconds to label: ", end - start)

Seconds to label:  126.90570259094238


# Steve Jobs

In [54]:
start = time.time()

In [83]:
widget, result = show_query(520, 0.2, 50, 5)

VGridWidget(jsglobals={'schema': [['Identity', ['id', 'name']], ['Genre', ['id', 'name']], ['Director', ['id',…

In [56]:
selected_segments_jobs = [
    (result['result'][i]['elements'][0]['min_frame'], result['result'][i]['elements'][0]['min_frame'])
    for i in widget.selected
]
print(selected_segments_jobs)

[(18839, 18839), (33234, 33234), (56701, 56701), (58482, 58482), (30592, 30592), (100750, 100750), (99826, 99826), (111112, 111112), (127182, 127182), (131931, 131931), (149091, 149091), (160355, 160355), (162592, 162592)]


In [84]:
convert_segments(selected_segments_jobs)

[(18839, 18976), (33234, 33284), (56701, 56819), (58482, 58585), (30592, 30615), (100750, 100856), (99826, 100209), (111112, 111138), (127182, 127311), (131931, 132087), (149091, 149274), (160355, 160580), (162592, 162653)]


In [57]:
end = time.time()

In [58]:
print("Seconds to label: ", end - start)

Seconds to label:  78.31287026405334


# Guardians of the Galaxy

In [59]:
start = time.time()

In [85]:
widget, result = show_query(74, 0.2, 50, 5)

VGridWidget(jsglobals={'schema': [['Identity', ['id', 'name']], ['Genre', ['id', 'name']], ['Director', ['id',…

In [61]:
selected_segments_gotg = [
    (result['result'][i]['elements'][0]['min_frame'], result['result'][i]['elements'][0]['min_frame'])
    for i in widget.selected
]
print(selected_segments_gotg)

[(27614, 27614), (28465, 28465), (54306, 54306), (68714, 68714), (118456, 118456), (146565, 146565), (151634, 151634), (157975, 157975), (156848, 156848), (26654, 26654), (23276, 23276), (29433, 29433)]


In [86]:
convert_segments(selected_segments_gotg)

[(27614, 27647), (28465, 28503), (54306, 54334), (68714, 68745), (118456, 118520), (146565, 146602), (151634, 151677), (157975, 158020), (156848, 156884), (26654, 26666), (23276, 23338), (29433, 29463)]


In [62]:
end = time.time()

In [63]:
print("Seconds to label: ", end - start)

Seconds to label:  51.962666034698486


# Daddy's Home

In [64]:
start = time.time()

In [87]:
widget, result = show_query(334, 0.2, 50, 5)

VGridWidget(jsglobals={'schema': [['Identity', ['id', 'name']], ['Genre', ['id', 'name']], ['Director', ['id',…

In [66]:
selected_segments_daddy = [
    (result['result'][i]['elements'][0]['min_frame'], result['result'][i]['elements'][0]['min_frame'])
    for i in widget.selected
]
print(selected_segments_daddy)

[(6664, 6664), (19173, 19173), (29445, 29445), (33339, 33339), (43067, 43067), (52057, 52057), (52717, 52717), (55006, 55006), (57986, 57986), (58889, 58889), (78143, 78143), (91840, 91840), (97268, 97268), (116269, 116269), (130170, 130170)]


In [88]:
convert_segments(selected_segments_daddy)

[(6664, 6732), (19173, 19204), (29445, 29477), (33339, 33419), (43067, 43084), (52057, 52125), (52717, 52796), (55006, 55032), (57986, 58007), (58889, 58935), (78143, 78242), (91840, 91889), (97268, 97345), (116269, 116424), (130170, 130269)]


In [67]:
end = time.time()

In [68]:
print("Seconds to label: ", end - start)

Seconds to label:  54.389283418655396


# Batman v Superman

In [69]:
start = time.time()

In [89]:
widget, result = show_query(299, 0.2, 50, 5)

VGridWidget(jsglobals={'schema': [['Identity', ['id', 'name']], ['Genre', ['id', 'name']], ['Director', ['id',…

In [71]:
selected_segments_bvs = [
    (result['result'][i]['elements'][0]['min_frame'], result['result'][i]['elements'][0]['min_frame'])
    for i in widget.selected
]
print(selected_segments_bvs)

[(12964, 12964), (19978, 19978), (38508, 38508), (44408, 44408), (51466, 51466), (52482, 52482), (57543, 57543), (62118, 62118), (69073, 69073), (82639, 82639), (100062, 100062), (127075, 127075), (188936, 188936), (202577, 202577)]


In [90]:
convert_segments(selected_segments_bvs)

[(12964, 13001), (19978, 20018), (38508, 38715), (44408, 44498), (51466, 51555), (52482, 52531), (57543, 57585), (62118, 62236), (69073, 69173), (82639, 82703), (100062, 100085), (127075, 127154), (188936, 188996), (202577, 202645)]


In [74]:
end = time.time()

In [75]:
print("Seconds to label: ", end - start)

Seconds to label:  93.14249897003174
